In [ ]:
#imports
import tomllib
import numpy as np
from pprint import pprint
import librosa
import requests
from pydub import AudioSegment
from io import BytesIO
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
with open("cfg.toml", "rb") as cfg:
    keys = tomllib.load(cfg)["spotify"]
    c_id = keys["client_id"]
    c_secret = keys["client_secret"]
    auth_manager = SpotifyClientCredentials(client_id=c_id, client_secret=c_secret)
    
sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
#taken from matthew baleanu and Mohamad-Hassan Bahsoun
#16khz downsample (default 22khz, spotify audio is 44k.)
#main audio loading
def downsampleAudio(audio_file_path, target_sample_rate=16000):
    #load the audio and downsample it. 
    signal, sampling_rate = librosa.load(audio_file_path, sr=target_sample_rate, mono=True)
    #normalize amplitude of the audio signal
    signal = signal / np.max(np.abs(signal))
    return signal, sampling_rate

#STFT the signal
def processAudio(audio_file_path):
    signal, sampling_rate = downsampleAudio(audio_file_path)

    #stft signal for feature computation
    stft_signal = librosa.stft(signal)
    return signal, stft_signal

In [ ]:
def computeRMS(signal):
    pass
    #return rms

def computeDynamicRange(signal):
    pass
    #return dynamic_range

def computeBPM(signal):
    pass
    #return bpm